## Import package & pre-generate hdf5 

In [6]:
import numpy as np
import h5py
import cv2
from train.get_model import ALLModel
import json
import h5py
import pickle
import scipy.io as sio
import matplotlib.pyplot as plt
import random
import tensorflow as tf
tf.random.set_seed(33)
from tensorflow.keras import backend as K
# from tensorflow import keras
from tensorflow.keras import layers,models,optimizers,callbacks,constraints
from tensorflow.keras.constraints import Constraint
from tensorflow.keras.layers import Input, Dense, Conv2D,multiply,Lambda,Add,Concatenate, Multiply,Conv2DTranspose,Layer, Reshape, ZeroPadding2D,Flatten, MaxPooling2D, UpSampling2D, BatchNormalization,LeakyReLU
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping,Callback,LearningRateScheduler,ModelCheckpoint
# set_session(tf.Session(config=config))
from sklearn.utils import shuffle
import math
import os
from tensorflow.keras import regularizers

####################create a subset from the whole hdf5 file, do not run if you don't need####################
# path='C:\\Users\\yidaiyao\\Desktop\\decoding_model\\data_nv30364657.hdf5'
# ff =h5py.File( path,'r')
# names=list(ff.keys())

# new_file='data/part_data.hdf5'
# with h5py.File(new_file, 'w', libver='latest', swmr=True) as fil:
#     fil.swmr_mode = True
    
#     for f in names:
#         group_n = fil.create_group(f)
        
#         temp=list(ff[f].keys())[:1]
        
#         for idx,t in enumerate(temp,0):
#                 data=ff[f][t]
#                 group_n.create_dataset('trial_%04i' % int(idx), data=np.array(data), dtype='float32')
########################################################################################################
path='data/part_data.hdf5'
ff =h5py.File( path,'r')
ff.keys()

<KeysViewHDF5 ['images_30', 'images_36', 'images_46', 'images_57', 'labels_30', 'labels_36', 'labels_46', 'labels_57']>

## parameter initialzation

In [2]:
json_path = 'train/parameter_init.json'

with open(json_path, "r") as f:
    param = json.load(f)
#####selsct set of parameter id####
if param['grid_search'] == True:
    jobid =os.getenv('SLURM_ARRAY_TASK_ID')
    jobid= int(jobid)
else:
    jobid = int(0)
####parameter initialization######
background = param['contrained_dim'][jobid]
udim = param['unsupervised_dim'][jobid]  ###
sdim = param['label_dim'][jobid]
bac = background
latent_dim = background + udim + sdim

alpha = param['alpha'][jobid]  ###
beta = param['beta'][jobid]  ####
theta = param['theta'][jobid]  ####
ks = param['ks'][jobid]  ####
flag = param['flag'][jobid]  ####
hdf5_file = param['hdf5_file']
batch_size=param['batch_size'][0]
epoch_num=param['epoch'][0]
image_name=param['image_name']
label_name=param['label_name']
img_x,img_y,img_z=param['image_size']
#############


## Load pre-trained model

In [3]:
model_str=ALLModel(img_x,img_y,img_z,sdim,udim,ks,flag,theta,alpha,beta,latent_dim,bac).get_model()
model_str.summary()
model_str.load_weights('xxx.h5')

Instructions for updating:
Colocations handled automatically by placer.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
mice1 (InputLayer)              [(None, 128, 128, 2) 0                                            
__________________________________________________________________________________________________
micelabel1 (InputLayer)         [(None, 5)]          0                                            
__________________________________________________________________________________________________
encoder (Encoder)               [(None, 5), (None, 2 4489729     mice1[0][0]                      
__________________________________________________________________________________________________
get_latent (get_latent)         ([(None, 9), (None,  30          micelabel1[0][0]                 
                      

## Check the reconstruction performance

In [ ]:
intermediate_layer_model = Model(inputs=allmodel.input,
                                 outputs=allmodel.output)###41 latent 42 label 43 image
imageout=np.zeros((4*4*189,128,128,2))
laaa=np.zeros((4*4*189,128,128,2))


labelout=np.zeros((4*4*189,5))
lbbb=np.zeros((4*4*189,5))
# latent=np.zeros((4*4*189,11))
# laaa=np.zeros((4*4*189,11))
image_name=['images_30', 'images_36', 'images_46', 'images_57']
label_name=['labels_30', 'labels_36', 'labels_46', 'labels_57']



idx={i:[] for i in range (4*4*189)}
z=0
trails=['trial_0027', 'trial_0127', 'trial_0227', 'trial_0327']
for i in range(4):
#     print(i)
    for j in trails:
        for k in range (189):
            idx[z].append(i)
            idx[z].append(j)
            idx[z].append(k)

            z+=1

z=0
ind=idx
for alp in range (4*4*189):
    i=ind[alp][0]
    j=ind[alp][1]
    k=ind[alp][2]
    
    im=np.rollaxis(np.expand_dims(np.array(a[image_name[i]][j][k]),axis=0).T,3,0)
#             print(np.rollaxis(im.T,3,0).shape)
    lab=np.array(a[label_name[i]][j][k])
    lab=lab.reshape(1,-1)
#             print(lab.shape)
    intermediate_output,pred_label = intermediate_layer_model.predict([np.rollaxis(im,2,1)/255,lab])#
    imageout[z]=intermediate_output
    labelout[z]=pred_label
    lbbb[z,:]=lab
    laaa[z,:]=np.rollaxis(im,2,1)/255

    z+=1
print('Mse_error for each frame:',np.mean(((imageout-laaa))**2))
from sklearn.metrics import r2_score

for i in range (5):
    r=r2_score(labelout[:,i],lbbb[:,i])
    
    print('r2 for label reconstruction:'+str(i)+'=',r)

In [ ]:
from matplotlib.pyplot import cm
plt.figure(1)
plt.imshow(np.concatenate((laaa[189*12+50,:,:,0],laaa[189*12+50,:,:,1]),axis=1),cmap=cm.gray)#; plt.show()
plt.title('Original Frame')
plt.figure(2)

plt.imshow(np.concatenate((imageout[189*12+50,:,:,0],imageout[189*12+50,:,:,1]),axis=1),cmap=cm.gray)#; plt.show()
plt.title('Reconstruction Frame')

## Get latent and save the corresponding images

In [8]:
intermediate_layer_model = Model(inputs=model_str.input,
                                 outputs=model_str.layers[3].output)###40 label predict, 41 image####40 back ####42
intermediate_layer_model .summary()

t=4
laaa=np.zeros((4*t*189,128,128,2))

latent=np.zeros((4*t*189,9))
# laaa=np.zeros((4*4*189,10))
image_name=['images_30', 'images_36', 'images_46', 'images_57']
label_name=['labels_30', 'labels_36', 'labels_46', 'labels_57']



idx={i:[] for i in range (4*388*189)}
z=0
trails=['trial_0300', 'trial_0040', 'trial_0219', 'trial_0128']


for i in range(4):
#     print(i)
    for j in trails:
        for k in range (189):
            idx[z].append(i)
            idx[z].append(j)
            idx[z].append(k)

            z+=1

z=0
ind=idx
for alp in range (4*t*189):
    i=ind[alp][0]
    j=ind[alp][1]
    k=ind[alp][2]
    
    im=np.rollaxis(np.expand_dims(np.array(a[image_name[i]][j][k]),axis=0).T,3,0)/255

    lab=np.array(a[label_name[i]][j][k])
    lab=lab.reshape(1,-1)
#             print(lab.shape)
    intermediate_output,_ = intermediate_layer_model.predict([np.rollaxis(im,2,1),lab])#
    latent[z]=intermediate_output
    laaa[z,:]=np.rollaxis(im,2,1)/255

    z+=1

np.save('LATENT44',latent)
np.save('IMAGEs44',laaa)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
mice1 (InputLayer)              [(None, 128, 128, 2) 0                                            
__________________________________________________________________________________________________
micelabel1 (InputLayer)         [(None, 5)]          0                                            
__________________________________________________________________________________________________
encoder (Encoder)               [(None, 5), (None, 2 4489729     mice1[0][0]                      
__________________________________________________________________________________________________
get_latent (get_latent)         ([(None, 9), (None,  30          micelabel1[0][0]                 
                                                                 encoder[0][0]              